# Momentum
- https://datasirup.tistory.com/109

In [19]:
import numpy as np
import tensorflow as tf
import pandas as pd
import logging, os
logging.disable(logging.WARNING)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

# 데이터를 전처리하는 함수

def sequences_shaping(sequences, dimension):
    
    results = np.zeros((len(sequences), dimension))
    for i, word_indices in enumerate(sequences):
        results[i, word_indices] = 1.0 
    
    return results
    
'''
1. 모멘텀(momentum)을 적용/비적용 할 하나의 모델을 자유롭게 생성합니다.
'''
    
def Momentum_model(word_num):
    
    model = tf.keras.Sequential([tf.keras.layers.Dense(32, input_shape=(word_num,), activation=tf.nn.relu),
                                 tf.keras.layers.Dense(32, activation='relu'),
                                 tf.keras.layers.Dense(1, activation='sigmoid')])
    
    return model

'''
2. 두 모델을 불러온 후 학습시키고 테스트 데이터에 대해 평가합니다.

   Step01. Momentum_model 함수를 이용해 
           두 모델을 불러옵니다. 모두 동일한 모델입니다.
   
   Step02. 두 모델의 손실 함수, 최적화 알고리즘, 
           평가 방법을 설정합니다.
   
   Step03. 두 모델의 구조를 확인하는 코드를 작성합니다.
   
   Step04. 두 모델을 각각 학습시킵니다. 
           검증용 데이터도 설정해주세요.
           두 모델 모두 'epochs'는 20, 'batch_size'는
           500으로 설정합니다.
   
   Step05. 학습된 두 모델을 테스트하고 
           binary crossentropy 값을 출력합니다. 
           둘 중 어느 모델의 성능이 더 좋은지 확인해보세요.
'''
    
word_num = 100
data_num = 25000

# Keras에 내장되어 있는 imdb 데이터 세트를 불러오고 전처리합니다.

(train_data, train_labels), (test_data, test_labels) = tf.keras.datasets.imdb.load_data(num_words = word_num)

print(train_data.shape)
print(train_data)
train_data = sequences_shaping(train_data, dimension = word_num)
test_data = sequences_shaping(test_data, dimension = word_num)
print(train_data.shape)
print(train_data)

sgd_model = Momentum_model(word_num)   # 모멘텀을 사용하지 않을 모델입니다.
msgd_model = Momentum_model(word_num)  # 모멘텀을 사용할 모델입니다.

sgd_opt = tf.keras.optimizers.SGD(0.01, momentum=0.0)
sgd_model.compile(loss='binary_crossentropy', optimizer=sgd_opt, metrics=['accuracy', 'binary_crossentropy'])

msgd_opt = tf.keras.optimizers.SGD(0.01, momentum=0.9)
msgd_model.compile(loss='binary_crossentropy', optimizer=msgd_opt, metrics=['accuracy', 'binary_crossentropy'])

sgd_model.summary()
msgd_model.summary()

# shape을 맞춰주는 과정
train_labels_df = pd.DataFrame(train_labels)
test_labels_df = pd.DataFrame(test_labels)

sgd_history = sgd_model.fit(train_data, train_labels_df, epochs=20, batch_size=500, validation_data=(test_data, test_labels_df), verbose=0)
print('\n')
msgd_history = msgd_model.fit(train_data, train_labels_df, epochs=20, batch_size=500, validation_data=(test_data, test_labels_df), verbose=0)

scores_sgd = sgd_model.evaluate(test_data, test_labels_df)
scores_msgd = msgd_model.evaluate(test_data, test_labels_df)

print('\nscores_sgd: ', scores_sgd[-1])
print('scores_msgd: ', scores_msgd[-1])

(25000,)
[list([1, 14, 22, 16, 43, 2, 2, 2, 2, 65, 2, 2, 66, 2, 4, 2, 36, 2, 5, 25, 2, 43, 2, 2, 50, 2, 2, 9, 35, 2, 2, 5, 2, 4, 2, 2, 2, 2, 2, 2, 39, 4, 2, 2, 2, 17, 2, 38, 13, 2, 4, 2, 50, 16, 6, 2, 2, 19, 14, 22, 4, 2, 2, 2, 4, 22, 71, 87, 12, 16, 43, 2, 38, 76, 15, 13, 2, 4, 22, 17, 2, 17, 12, 16, 2, 18, 2, 5, 62, 2, 12, 8, 2, 8, 2, 5, 4, 2, 2, 16, 2, 66, 2, 33, 4, 2, 12, 16, 38, 2, 5, 25, 2, 51, 36, 2, 48, 25, 2, 33, 6, 22, 12, 2, 28, 77, 52, 5, 14, 2, 16, 82, 2, 8, 4, 2, 2, 2, 15, 2, 4, 2, 7, 2, 5, 2, 36, 71, 43, 2, 2, 26, 2, 2, 46, 7, 4, 2, 2, 13, 2, 88, 4, 2, 15, 2, 98, 32, 2, 56, 26, 2, 6, 2, 2, 18, 4, 2, 22, 21, 2, 2, 26, 2, 5, 2, 30, 2, 18, 51, 36, 28, 2, 92, 25, 2, 4, 2, 65, 16, 38, 2, 88, 12, 16, 2, 5, 16, 2, 2, 2, 32, 15, 16, 2, 19, 2, 32])
 list([1, 2, 2, 2, 2, 78, 2, 5, 6, 2, 2, 2, 2, 26, 4, 2, 8, 2, 2, 14, 2, 20, 13, 2, 2, 2, 2, 5, 2, 2, 2, 21, 14, 69, 2, 8, 30, 23, 7, 4, 2, 2, 93, 4, 2, 9, 2, 2, 5, 2, 4, 2, 9, 35, 2, 4, 2, 9, 2, 2, 4, 2, 9, 4, 2, 2, 19, 4, 2, 5, 89, 2

Model: "sequential_26"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_80 (Dense)                │ (None, 32)             │         3,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_81 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_82 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,321 (16.88 KB)

 Trainable params: 4,321 (16.88 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_83 (Dense)                │ (None, 32)             │         3,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_84 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_85 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,321 (16.88 KB)

 Trainable params: 4,321 (16.88 KB)

 Non-trainable params: 0 (0.00 B)



782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 295us/step - accuracy: 0.6313 - binary_crossentropy: 0.6432 - loss: 0.6432
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 291us/step - accuracy: 0.7054 - binary_crossentropy: 0.5657 - loss: 0.5657

scores_sgd:  0.643226683139801
scores_msgd:  0.5645570158958435


In [ ]:
import numpy as np
import tensorflow as tf

import logging, os
logging.disable(logging.WARNING)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

# 데이터를 전처리하는 함수

def sequences_shaping(sequences, dimension):
    
    results = np.zeros((len(sequences), dimension))
    for i, word_indices in enumerate(sequences):
        results[i, word_indices] = 1.0 
    
    return results
    
'''
1. 모멘텀(momentum)을 적용/비적용 할 하나의 모델을 자유롭게 생성합니다.
'''
    
def Momentum_model(word_num):
    
    model = tf.keras.Sequential([None])
    
    return model

'''
2. 두 모델을 불러온 후 학습시키고 테스트 데이터에 대해 평가합니다.

   Step01. Momentum_model 함수를 이용해 
           두 모델을 불러옵니다. 모두 동일한 모델입니다.
   
   Step02. 두 모델의 손실 함수, 최적화 알고리즘, 
           평가 방법을 설정합니다.
   
   Step03. 두 모델의 구조를 확인하는 코드를 작성합니다.
   
   Step04. 두 모델을 각각 학습시킵니다. 
           검증용 데이터도 설정해주세요.
           두 모델 모두 'epochs'는 20, 'batch_size'는
           500으로 설정합니다.
   
   Step05. 학습된 두 모델을 테스트하고 
           binary crossentropy 값을 출력합니다. 
           둘 중 어느 모델의 성능이 더 좋은지 확인해보세요.
'''
    
word_num = 100
data_num = 25000

# Keras에 내장되어 있는 imdb 데이터 세트를 불러오고 전처리합니다.

(train_data, train_labels), (test_data, test_labels) = tf.keras.datasets.imdb.load_data(num_words = word_num)

train_data = sequences_shaping(train_data, dimension = word_num)
test_data = sequences_shaping(test_data, dimension = word_num)

sgd_model = None   # 모멘텀을 사용하지 않을 모델입니다.
msgd_model = None  # 모멘텀을 사용할 모델입니다.

sgd_opt = None
sgd_model.compile(None)

msgd_opt = None
msgd_model.compile(None)

None
None

sgd_history = None
print('\n')
msgd_history = None

scores_sgd = None
scores_msgd = None

print('\nscores_sgd: ', scores_sgd[-1])
print('scores_msgd: ', scores_msgd[-1])